In [1]:
!pip install tensorflow tensorflow-gpu pandas matplotlib sklearn

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [15 lines of output]
  The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
  rather than 'sklearn' for pip commands.
  
  Here is how to fix this error in the main use cases:
  - use 'pip install scikit-learn' rather than 'pip install sklearn'
  - replace 'sklearn' by 'scikit-learn' in your pip requirements files
    (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
  - if the 'sklearn' package is used by one of your dependencies,
    it would be great if you take some time to track which package uses
    'sklearn' instead of 'scikit-learn' and report it to their issue tracker
  - as a last resort, set the environment variable
    SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
  
  More information is available at
  https://github.com/scikit-learn/sklearn-pypi-package
  [end of output]
  
  note: This error originates from a subpr

In [42]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

In [43]:
df = pd.read_csv(
    os.path.join("jigsaw-toxic-comment-classification-challenge","train.csv", "train.csv"))

In [44]:
df = df.iloc[:85000]

In [45]:
df.shape

(85000, 8)

In [46]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [47]:
from tensorflow.keras.layers import TextVectorization 

In [48]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [49]:
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [50]:
df.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [51]:
MAX_WORDS = 200000 #number of words in vocab

In [52]:
vectorizer = TextVectorization(max_tokens=MAX_WORDS,
                              output_sequence_length=1800,
                              output_mode='int')

In [53]:
vectorizer.adapt(X.values)

In [54]:
vectorizer("hello words life is great")

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([289, 395, 291, ...,   0,   0,   0], dtype=int64)>

In [55]:
vectorized_text = vectorizer(X.values)

In [56]:
vectorized_text

<tf.Tensor: shape=(85000, 1800), dtype=int64, numpy=
array([[   658,     76,      2, ...,      0,      0,      0],
       [144899,     54,   2526, ...,      0,      0,      0],
       [   397,    434,     70, ...,      0,      0,      0],
       ...,
       [    12,      9,      2, ...,      0,      0,      0],
       [   172,      4,    676, ...,      0,      0,      0],
       [  3207,      2,     24, ...,      0,      0,      0]], dtype=int64)>

In [57]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(60000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)


In [58]:
batch_x, batch_y = dataset.as_numpy_iterator().next()

In [59]:
batch_x.shape

(16, 1800)

In [60]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [61]:
train_generator = train.as_numpy_iterator()

In [62]:
train_generator.next()

(array([[ 123,    7,   13, ...,    0,    0,    0],
        [1142,  849,  149, ...,    0,    0,    0],
        [   5,  798,    8, ...,    0,    0,    0],
        ...,
        [ 889,  214,   96, ...,    0,    0,    0],
        [1542,    2,  121, ...,    0,    0,    0],
        [ 610,   16,   38, ...,    0,    0,    0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

In [63]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [64]:
model = Sequential()
model.add(Embedding(MAX_WORDS+1, 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='sigmoid'))

In [65]:
model.compile(loss='BinaryCrossentropy', optimizer='adam')

In [66]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [67]:
history = model.fit(train, epochs=1, validation_data=val)

3719/3719 ━━━━━━━━━━━━━━━━━━━━ 5050s 1s/step - loss: 0.1020 - val_loss: 0.0492


In [68]:
model.save("85k_model.h5")

In [26]:
# from tensorflow.keras.models import load_model

In [69]:
# model = load_model('./toxicity.h5')

In [70]:
from matplotlib import pyplot as plt

In [71]:
input_text = vectorizer("you freaking suck!")

In [72]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [73]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [74]:
model.predict(batch_X)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


array([[7.03835934e-02, 8.73291923e-04, 1.15038976e-02, 2.98080174e-03,
        9.23130289e-03, 3.82113154e-03],
       [1.89539592e-03, 6.93696563e-07, 1.13787268e-04, 9.37698496e-06,
        5.17149019e-05, 1.21437870e-05],
       [1.39698442e-02, 3.27936104e-05, 1.34031230e-03, 2.13406980e-04,
        8.61006440e-04, 2.74691847e-04],
       [5.06971665e-02, 4.28071507e-04, 7.21685681e-03, 1.69011171e-03,
        5.55047719e-03, 2.16222834e-03],
       [4.80440110e-02, 4.08262946e-04, 6.98112510e-03, 1.63941714e-03,
        5.32278791e-03, 2.08729575e-03],
       [3.39490622e-02, 2.01943170e-04, 4.41304129e-03, 9.31104878e-04,
        3.18125403e-03, 1.18730753e-03],
       [9.70469415e-03, 1.84474120e-05, 9.36144206e-04, 1.34709611e-04,
        5.56346204e-04, 1.72709173e-04],
       [7.03189671e-02, 8.32181249e-04, 1.11877443e-02, 2.85514118e-03,
        8.93720239e-03, 3.65617755e-03],
       [7.22949812e-03, 1.11912232e-05, 6.87551859e-04, 9.07679569e-05,
        3.80475685e-04, 

In [131]:
# !pip install gradio jinja2

In [75]:
import gradio as gr


In [76]:
import tensorflow as tf

In [77]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [78]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

In [79]:
interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://aeb02351d292628e4b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 988ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
Created dataset file at: .gradio\flagged\dataset2.csv
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step
